# Классификация отзывов

## Данные

Скачиваются по [ссылке](https://drive.google.com/open?id=1gJEpwM3zdV5xe9nRU0VwQmGTiOTQdp10). Это отзывы пользователей на фильмы с КиноПоиска.

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('reviews.csv')

In [8]:
df.head()

,label,text
0,1,Эпиграф Добро которое ты делаешь от сердца ты ...
1,1,Теперь это один из моих любимых фильмов в жанр...
2,1,Что скрыто в фильме Лучше не бывает Одна шикар...
3,1,Перед нами очень милое и доброе кино которое л...
4,1,Завязка Мелвин Удал популярный писатель Нет не...


In [9]:
set(df.label)

{-1, 0, 1}

В колонке `label` три класса: положительный (1), нейтральный (0) и отрицательный (-1).

## Baseline

In [23]:
!pip install sklearn

Processing /home/maryszmary/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.22.2.post1-cp36-cp36m-manylinux1_x86_64.whl (7.1 MB)
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)


In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
vec = CountVectorizer()
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.label)

In [12]:
nb = MultinomialNB()
clf = nb.fit(X_train, y_train)

In [25]:
from sklearn.metrics import classification_report

In [14]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.69      0.21      0.32      1067
           0       0.35      0.08      0.13      1369
           1       0.82      0.98      0.90      9190

    accuracy                           0.81     11626
   macro avg       0.62      0.42      0.45     11626
weighted avg       0.75      0.81      0.75     11626



## Задание

Побейте baseline выше как минимум тремя способами.


**Что стоит сделать**:
* попробовать разный препроцессинг
* попробовать разные классификаторы
* поподбирать гиперпараметры классификаторов, можно с использованием GreedSearch
* попробуйте объяснить полученные результаты
* визуализируйте веса признаков для лучшей модели (если лучшая модель не выдаёт коэффициенты признаков, возьмите логистическую регрессию); какие признаки оказались самыми значимыми?


Good luck and have fun! ;)

### С помощью w2v

In [11]:
from string import punctuation

In [12]:
sentences = [sent.lower().strip(punctuation).split() for sent in df.text]

In [15]:
model_news = word2vec.Word2Vec(sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

In [16]:
model_news.most_similar('добро')

/home/maryszmary/.local/share/virtualenvs/NLP_DPO_2020-U6tis9ju/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('зло', 0.8270263075828552),
 ('побеждает', 0.6942523717880249),
 ('пожаловать', 0.6580352783203125),
 ('торжествует', 0.6507160663604736),
 ('добра', 0.6271214485168457),
 ('зла', 0.5899354219436646),
 ('победит', 0.5741604566574097),
 ('восторжествует', 0.5713249444961548),
 ('милосердие', 0.5652226209640503),
 ('справедливость', 0.554529070854187)]

In [19]:
def get_text_embedding(text):
    tokens = text.lower().strip(punctuation).split()
    result_embeddings = [model_news[token] for token in tokens if token in model_news] 
    return sum(result_embeddings) / len(result_embeddings)

In [20]:
text_embeddings = [get_text_embedding(text) for text in df.text]

/home/maryszmary/.local/share/virtualenvs/NLP_DPO_2020-U6tis9ju/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/maryszmary/.local/share/virtualenvs/NLP_DPO_2020-U6tis9ju/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
len(text_embeddings)

46501

In [26]:
X_train, X_test, y_train, y_test = train_test_split(text_embeddings, df.label)

In [29]:
from sklearn.linear_model import LogisticRegression

In [31]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/home/maryszmary/.local/share/virtualenvs/NLP_DPO_2020-U6tis9ju/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
print(classification_report(y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.60      0.47      0.53      1125
           0       0.42      0.12      0.18      1412
           1       0.85      0.97      0.91      9089

    accuracy                           0.82     11626
   macro avg       0.62      0.52      0.54     11626
weighted avg       0.77      0.82      0.78     11626

